In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
os.chdir(directory['curr'])

*Specify settings:*

In [2]:
name = 'A3'
# catalogue = 'dors_2'
catalogue = 'C2'
f = {'tech': directory['data']+f"/techdata_{catalogue}.xlsx",
     'other': directory['data']+f"/Othdata_{catalogue}.xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

## 1: Initialize model

*Initialize with ID and EOP technologies:*

In [5]:
gm = abatement.abate(nts=nts,tech=tech,work_folder=directory['work'],use_EOP=False,**{'data_folder':directory['gams'],'name': name})
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False) 

*Provide initial values:*

In [6]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

## 2: Solve model

In [7]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as f:
    kwargs_mi = {'opt_file': f.read()}

*Debugging:*

In [8]:
gm.model.database['testminobj'] = 0;

#### 2.1: EOP

*Set up baseline model and solve:*

In [9]:
gm.write_and_run(name='EOP',kwargs_mi=kwargs_mi)
db_EOP = gm.model_instances['EOP'].out_db

*Read back solution to main db:*

In [10]:
gm.model.database.merge_dbs(gm.model.database,db_EOP,'second')

#### 2.2: ID calibrate

In [11]:
gm.reset_settings()
gm.setstate('ID_calibrate',init=False)
gm.write_and_run(name='ID_calibrate',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)
db_ID_C = gm.model_instances['ID_calibrate'].out_db

#### 2.3: EOP calibrate

*Change to EOP calibration state and resolve:*

In [12]:
gm.reset_settings()
gm.setstate('EOP_calibrate',init=False)
gm.write_and_run(name='EOP_calibrate',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)
db_EOP_C = gm.model_instances['EOP_calibrate'].out_db

## A: Sneaky solve with non-default solve statement

*Reset to EOP state:*

In [13]:
gm.reset_settings()
gm.setstate('EOP',init=False)
gm.write_and_run(name='EOP',kwargs_init={'check_variables':True},overwrite=True,kwargs_mi=kwargs_mi)

*set up target database:*

In [14]:
db_star = DataBase.GPM_database()
db_star['sigmaG'] = gm.get('sigmaG')/10
db_star['sigma'] = pd.Series(5, index = gm.get('kno_ID_CU'),name='sigma')

In [15]:
gm.model_instances['EOP'].solve_sneakily(db_star=db_star)

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [16]:
db = gm.model_instances['EOP'].out_db

*Solve sneakily:*

In [17]:
# gm.model_instances['EOP_calibrate'].solve_sneakily(db_star=db_star, kwargs_shock = {'solvetext': gm.model.settings.get_conf('solve')})

*Set up baseline model and solve:*

In [18]:
# gm.write_and_run(name='EOP',kwargs_mi=kwargs_mi)
# db_EOP = gm.model_instances['EOP'].out_db